In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import struct
import json

# Load dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Separate features and labels
X_train = train_df.drop(columns=['Activity', 'subject'])
y_train = train_df['Activity']
X_test = test_df.drop(columns=['Activity', 'subject'])
y_test = test_df['Activity']

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define model: 561 inputs -> 128 -> 64 -> 6 outputs
model = Sequential([
    Dense(128, activation='relu', input_shape=(561,)),
    Dense(64, activation='relu'),
    Dense(6, activation='softmax')
])

# Compile and train
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test_encoded))

# Evaluate
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Save weights and biases
weights = []
biases = []
for layer in model.layers:
    w, b = layer.get_weights()
    weights.append(w)
    biases.append(b)

# Write weights to binary file
with open('weights.bin', 'wb') as f:
    for w, b in zip(weights, biases):
        # Write weights (row-major)
        for row in w:
            for val in row:
                f.write(struct.pack('f', val))
        # Write biases
        for val in b:
            f.write(struct.pack('f', val))

# Save configuration
config = {
    'layer_sizes': [561, 128, 64, 6],
    'activations': ['relu', 'relu', 'softmax'],
    'num_layers': 4
}
with open('model_config.json', 'w') as f:
    json.dump(config, f)

# Save 32 test samples
num_samples = 2500
with open('sample_inputs.bin', 'wb') as f:
    f.write(X_test_scaled[:num_samples].astype(np.float32).tobytes())
with open('sample_labels.bin', 'wb') as f:
    f.write(y_test_encoded[:num_samples].astype(np.int32).tobytes())

print("Weights, config, and sample data saved.")

# Load the 32 test samples and labels from the saved files
sample_inputs = np.fromfile('sample_inputs.bin', dtype=np.float32).reshape(num_samples, -1)
sample_labels = np.fromfile('sample_labels.bin', dtype=np.int32)

# test on those samples
predictions = model.predict(sample_inputs)
predicted_labels = np.argmax(predictions, axis=1)
# Compare with actual labels
correct_predictions = np.sum(predicted_labels == sample_labels)
accuracy = correct_predictions / len(sample_labels)
print(f"Accuracy on random samples: {accuracy:.4f}")
# print shape of data
print(f"Shape of sample inputs: {sample_inputs.shape}")

Epoch 1/10


/home/aditya-trivedi/thats_me/NN/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8255 - loss: 0.4567 - val_accuracy: 0.9399 - val_loss: 0.1629
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9695 - loss: 0.0796 - val_accuracy: 0.9274 - val_loss: 0.1904
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9704 - loss: 0.0748 - val_accuracy: 0.9355 - val_loss: 0.1791
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9837 - loss: 0.0447 - val_accuracy: 0.9345 - val_loss: 0.2303
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9872 - loss: 0.0354 - val_accuracy: 0.9420 - val_loss: 0.1947
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9829 - loss: 0.0449 - val_accuracy: 0.9464 - val_loss: 0.1697
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9923 - loss: 0.0249 - val_accuracy: 0.9420 - val_loss: 0.2024
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9948 - loss: 0.0169 - val_accuracy: 0.9393 - val_